In [18]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import numpy as np
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain_qdrant import QdrantVectorStore
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
QDRANT_URL = "http://127.0.0.1:6333"
COLLECTION_NAME = "hybrid_docs"
DATA_PATH = "../data/processed/proc_docx.jsonl"
EMBED_MODEL_NAME = 'intfloat/multilingual-e5-large'
EMBED_DIM = 1024

In [3]:
docs = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        text = data.get("text", "")
        meta = {
            "doc_id": data.get("doc_id", ""),
            "file_path": data.get("file_path", "")
        }
        if text.strip():
            docs.append({"text": text, "metadata": meta})

In [4]:
len(docs)

31

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts, metadatas = [], []
for doc in docs:
    chunks = splitter.split_text(doc["text"])
    texts.extend(chunks)
    metadatas.extend([doc["metadata"]] * len(chunks))

In [6]:
len(texts), len(metadatas)

(53, 53)

In [7]:
texts = ['passage: ' + t for t in texts]

In [15]:
documents = [
    Document(page_content=texts[i], metadata=metadatas[i])
    for i in range(len(texts))
]

In [20]:
# emb = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)
# embedder = SentenceTransformer(EMBED_MODEL_NAME)
embedder = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)

In [10]:
sparse_embedder = FastEmbedSparse(model_name="Qdrant/bm25")

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

dutch.txt:   0%|          | 0.00/453 [00:00<?, ?B/s]

danish.txt:   0%|          | 0.00/424 [00:00<?, ?B/s]

french.txt:   0%|          | 0.00/813 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

italian.txt: 0.00B [00:00, ?B/s]

romanian.txt: 0.00B [00:00, ?B/s]

spanish.txt: 0.00B [00:00, ?B/s]

norwegian.txt:   0%|          | 0.00/851 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [ ]:
qdrant = QdrantVectorStore.from_documents(
    documents,
    url=QDRANT_URL,
    embedding=embedder,
    sparse_embedding=sparse_embedder,
    # location=":memory:",
    collection_name=COLLECTION_NAME,
    retrieval_mode=RetrievalMode.HYBRID,
    # force_recreate=True
)

In [28]:
# query = "What did the president say about Ketanji Brown Jackson"
# found_docs = qdrant.similarity_search(query)
# found_docs

# query = "Итоги запусков кампаний по категории бытовой химии"
# query = "Успешные кампании кофе и чай"
query = "Итоги запусков кампаний по категории молочная продукция"
query = 'query: ' + query
found_docs = qdrant.similarity_search(query, k=3)
# for score, text in hybrid_search(query, alpha=0.8, top_k=5):
for f in found_docs:
    # print(f"{score:.3f} — {f.page_content}")
    print(f"{f.page_content}")
    print('-'*80)

passage: Настоящий отчет подготовлен по результатам акция в регионе Урал. Основной целью являлось повышение продаж и вовлеченности по категории Молочная продукция. Несмотря на корректную реализацию кампании, существенного роста ключевых показателей не зафиксировано. Кампания началась с оптимистичных ожиданий, однако уже в первые дни стало ясно, что уровень вовлеченности аудитории ниже прогнозируемого. Основные показатели остались на уровне контрольного периода, что указывает на слабое восприятие механики. Причинами неуспеха стали низкая узнаваемость, ограниченный бюджет на медийное продвижение и недостаточная персонализация предложений. Следует уделить внимание более точному позиционированию кампании и выбору каналов с наибольшим откликом. Несмотря на обширный охват, кампания не вызвала заметного увеличения количества визитов или среднего чека. Вероятно, потребители не восприняли предложение как достаточно ценное. Дополнительный анализ показал, что поведение покупателей оставалось стаб

In [31]:
qdrant.similarity_search_with_score(query, k=3)

[(Document(metadata={'doc_id': 7, 'file_path': './data/raw/Акция — Урал — Молочная продукция (февраль 2025).docx', '_id': '544ea6b4-fd90-4518-9a99-7444fae4d48b', '_collection_name': 'hybrid_docs'}, page_content='passage: Настоящий отчет подготовлен по результатам акция в регионе Урал. Основной целью являлось повышение продаж и вовлеченности по категории Молочная продукция. Несмотря на корректную реализацию кампании, существенного роста ключевых показателей не зафиксировано. Кампания началась с оптимистичных ожиданий, однако уже в первые дни стало ясно, что уровень вовлеченности аудитории ниже прогнозируемого. Основные показатели остались на уровне контрольного периода, что указывает на слабое восприятие механики. Причинами неуспеха стали низкая узнаваемость, ограниченный бюджет на медийное продвижение и недостаточная персонализация предложений. Следует уделить внимание более точному позиционированию кампании и выбору каналов с наибольшим откликом. Несмотря на обширный охват, кампания н

In [34]:
retriever = qdrant.as_retriever(search_kwargs={"k": 3})
docs = retriever.invoke(query)
for doc in docs:
    print(doc.metadata)

{'doc_id': 7, 'file_path': './data/raw/Акция — Урал — Молочная продукция (февраль 2025).docx', '_id': '544ea6b4-fd90-4518-9a99-7444fae4d48b', '_collection_name': 'hybrid_docs'}
{'doc_id': 5, 'file_path': './data/raw/Тестирование механик стимулирования продаж (апрель–июнь 2025).docx', '_id': 'c1386d84-8289-428d-8079-0bfdb0fef2ef', '_collection_name': 'hybrid_docs'}
{'doc_id': 14, 'file_path': './data/raw/Тестовая персонализация — ПФО — Семьи с детьми (январь 2025).docx', '_id': '34fb30b7-af6c-4f8d-87a1-2791515b2388', '_collection_name': 'hybrid_docs'}
